Reconstructing a lineage tree from mutation table

In [1]:
import os
import sys
sys.path.append('path_to_tmc_wrapper')

from tmc_wrapper.utils import to_dict_no_nans, memory_expensive_random_choose, run_command
from tmc_wrapper.TMC_formatting import map_cell_ids_for_sagi, format_triplet, convert_names_in_sagis_newick
from tmc_wrapper.triplet_scoring import choose_best_pair, NoIntersectionLociException

import pandas as pd

In [2]:
project_dir = 'example-dream-small'
aquisition_coverage_case = 'n-000001'

In [3]:
df = pd.read_csv(os.path.join(project_dir,aquisition_coverage_case,'mutation_table.csv'), index_col='names')

In [4]:
d = to_dict_no_nans(df)

In [5]:
triplets_file = '/tmp/triplets_test.txt'
cell_id_map_for_sagi = map_cell_ids_for_sagi(d)
with open(triplets_file, 'w') as f:
    for triplet in memory_expensive_random_choose(d.keys(), k=3, n=5000):
        try:
            pair, score = choose_best_pair(triplet, d)
        except NoIntersectionLociException:
            continue
        f.write(format_triplet(triplet, pair, score, cell_id_map_for_sagi, print_scores=True))

In [6]:
import os

class EmptyTripletsFile(Exception):
    pass


def run_TMC(triplets_file, output_newick):
    # tmc(
    #     "-fid", triplets_file,
    #     "-frtN", output_newick)
    if os.stat(triplets_file).st_size == 0:
        raise EmptyTripletsFile('Empty file: '.format(triplets_file))
    sagi_cli = 'full_path_to_tmc_wrapper/TMC/treeFromTriplets -fid {} -frtN {} -w 1 -index 2'.format(
        triplets_file, output_newick)
    return os.system(sagi_cli)

In [7]:
index_labeled_output_newick = '/tmp/index_labeled_triplets_test.newick'
run_TMC(triplets_file, index_labeled_output_newick)

256

In [8]:
newick_tree_path = '/tmp/triplets_test.newick'

In [9]:
convert_names_in_sagis_newick(index_labeled_output_newick, newick_tree_path, cell_id_map_for_sagi)


Evaluating the tree reconstruction agains a know reference using TreeCMP

In [10]:
import dendropy
tns = dendropy.TaxonNamespace()
ref_tree = dendropy.Tree.get_from_path(os.path.join(project_dir,'simulation.distance.newick'),'newick', taxon_namespace=tns, preserve_underscores=True)
tmc_rec_tree = dendropy.Tree.get_from_path('/tmp/triplets_test.newick',"newick",taxon_namespace=tns, preserve_underscores=True)

In [11]:
sim_tree = ref_tree
cat_nodes = dict()
for n in sim_tree.leaf_nodes():
    cat_nodes.setdefault(n.taxon.label.split('_')[1], list()).append(n.taxon)
sim_tree.prune_taxa(cat_nodes['Dead'])  # trim dead nodes from the reference
sim_tree.prune_taxa(cat_nodes['Null'])  # trim null nodes from the reference
sim_tree.write_to_path(os.path.join(project_dir,'simulation.distance.trimmed.newick'), 'newick')

In [12]:
def compare(path_simulation_newick, path_reconstructed_newick, path_score_output):

    path_treecmp = 'path_to_treecmp/bin/TreeCmp/bin/treeCmp.jar'

    #  Metrics for rooted trees
    metrics = {
        "rooted": "mc rc ns tt mp mt co",
        "unrooted": "ms rf pd qt um"
    }

    cmd = [
        'java', '-jar', path_treecmp,
        '-P', '-N', '-I'
        '-r', path_simulation_newick,
        '-i', path_reconstructed_newick,
        '-o', path_score_output,
        '-d'
    ]

    cmd.extend(metrics['rooted'].split(' '))

    run_command(cmd)
    
    
def get_scores(path_simulation_newick, path_reconstructed_newick, path_score_output):

    compare(path_simulation_newick, path_reconstructed_newick, path_score_output)

    df_metrics = pd.read_csv(
        path_score_output,
        sep='\t',
        nrows=1
    )

    return df_metrics

In [13]:
scores = get_scores(
    os.path.join(project_dir,'simulation.distance.trimmed.newick'),
    '/tmp/triplets_test.newick',
    '/tmp/scores.txt'
)

In [14]:
scores.T

,0
No,1.0000
RefTree,1.0000
Tree,1.0000
RefTree_taxa,55.0000
Tree_taxa,55.0000
Common_taxa,55.0000
MatchingCluster,238.0000
MatchingCluster_toYuleAvg,0.5959
MatchingCluster_toUnifAvg,0.4272
R-F_Cluster,35.5000


Directly calculating the number of correct triplets comparing to a reference tree

In [15]:
from tmc_wrapper.triplets_distance import triplets_score

In [16]:
counts = triplets_score(tmc_rec_tree, sim_tree, n=1000)

In [17]:
print("Percentage of correct triplets: {}%".format(100*counts[True]/(counts[True]+counts[False])))

Percentage of correct triplets: 68.2%
